In [3]:
from LinkGenerationEnv import LinkGenerationEnvV2_Continuous
import numpy as np
from Heuristics import *
from Util import execute, collect_episode


In [15]:
from Util import pick_actions_uniformly, bins, bin_probability, F

n_links, threshold, decay_rate, tradeoff, actions = 4, 0.5, 0.2, 0.7, [0.3, 0.7]

acts = []
for i in range(4):
    p = bin_probability(i, tradeoff, decay_rate, threshold, actions)
    acts.append([F(p,tradeoff),p])

for act in acts:
    print(act,bins(act[1], tradeoff, decay_rate, threshold) )
    # print(act[1])


In [17]:
from Util import bins, bin_probability

n_links, threshold, decay_rate, tradeoff, actions = 4, 0.5, 0.2, 0.7, [0.4, 0.7]
# n_links, threshold, decay_rate, tradeoff, actions = 6, 0.5, 0.08, 1, [0.2, 0.5]

env = LinkGenerationEnvV2_Continuous(n_links, threshold, decay_rate, tradeoff, actions)

heuristic_maker = lambda x: MatchingHeuristic(x, n_links, threshold, decay_rate, tradeoff, actions)
# ps = [0.499999, 0.47917723308126037, 0.45662128225204746,0.43218671241964884,0.40571705891601073,0.37704282558968244,0.34598039945177667,0.3123308749259747,0.27587878017376216,0.2363906973390281]
ps = [0.42067085700338963, 0.5386330365566891, 0.6352123006570822, 0.7]

shots = 2000
for n_links in range(1, 5):
    env = LinkGenerationEnvV2_Continuous(n_links, threshold, decay_rate, tradeoff, actions)

    for i in range(len(ps)):
        bin = bins(ps[i], tradeoff, decay_rate, threshold)
        if bin < n_links:
            continue

        mv_heuristic = heuristic_maker(ps[i])
        mv_total = execute(env, mv_heuristic, shots)
        print("MVHeuristic:", np.mean(mv_total), 3*np.std(mv_total)/np.sqrt(shots), bin)
    print("----------------")

In [5]:
from Util import bin_probability
bin_probability(6, tradeoff, decay_rate, threshold, actions)

In [16]:
# Parameter Tuning
n_links, threshold, decay_rate, tradeoff, actions = 6, 0.5, 0.08, 1, [0.2, 0.5]
env = LinkGenerationEnvV2_Continuous(n_links, threshold, decay_rate, tradeoff, actions)

import matplotlib.pyplot as plt 

def tune_probability(env, ps, heuristic_creator, n=20000):
    ys, ystds = [], []
    for p in ps:
        heuristic = heuristic_creator(p)
        returns = execute(env, heuristic, n)
        ys.append(np.mean(returns))
        ystds.append(np.std(returns))
    return ys, ystds

matching_heuristic_creator = lambda p: MatchingHeuristic(p, n_links, threshold, decay_rate, tradeoff, actions) 
sa_heuristic_creator = lambda p: SAHeuristic(p)

n = 20000
ps = np.linspace(actions[0], 0.345, 50)
ys, ystds = tune_probability(env, ps, sa_heuristic_creator,n)
yerrs = 3*np.array(ystds)/np.sqrt(n)

plt.errorbar(ps, np.abs(ys), yerr=yerrs)
